<a href="https://colab.research.google.com/github/Shreyas-Tayal/python-projects/blob/main/Brain_Tumor_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install keras_cv

In [ ]:
# @title
# Setting up the imports
import keras
from keras import layers
from keras import ops
import keras_cv
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# Setting up datasets
img_size = [256,256]

# Correcting the filepath
filepath_data = '/content/drive/MyDrive/Brain_Tumor_Data/Data' # Assuming 'Training' is the correct directory
filepath_ver = '/content/drive/MyDrive/Brain_Tumor_Data/Validation'

#tf.data.Dataset is a representation of the dataset (Generator)
main_dataset = keras.utils.image_dataset_from_directory(
    filepath_data,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=img_size
)

#print(len(main_dataset))

train_size = int(len(main_dataset)*0.8)
test_size = int(len(main_dataset)*0.2)

train_dataset = main_dataset.take(train_size)
test_dataset = main_dataset.skip(train_size).take(test_size)

#print(len(train_dataset))

# Turns the tf.data.Dataset into a numpy array
train_iterator = train_dataset.as_numpy_iterator()
test_iterator = test_dataset.as_numpy_iterator()

# Takes a batch from the array
train = train_iterator.next()
test = test_iterator.next()

#print(train[0].shape)

NotFoundError: Could not find directory /content/drive/MyDrive/Brain_Tumor_Data/Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_img, train_label = train

test_img, test_label = test

# Normalize training and testing data (We do this as the image input in the eval phase also has the same happen to it)
train_img = train_img / 255.0
test_img = test_img / 255.0
# The abover statements normalize the data in the range [0, 1]

print("train_labels shape:", train_label.shape)

if len(train_label) == 1:  # If labels are not one-hot encoded (add .shape if it doesnt work)
    train_label = keras.utils.to_categorical(train_label, num_classes=4)
    test_label = keras.utils.to_categorical(test_label, num_classes=4)

print("train_labels shape:", train_label.shape)

# Tensorboard and backup model setup
logdir = '/content/drive/MyDrive/Brain_Tumor_Data/logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
backup = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Brain_Tumor_Data/logs/backup_model.keras', verbose=2)


# Debugging tool
tf.debugging.experimental.enable_dump_debug_info(logdir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

# Input layer (batch_size here is unecessary, keras handles that by default)
img_input = keras.Input(shape=(256, 256, 3))
print("Input shape:", img_input.shape)

# Creates CNN layers
conv1 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img_input)
pooling1 = keras.layers.MaxPool2D()(conv1)

conv2 = keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(pooling1)
pooling2 = keras.layers.MaxPool2D()(conv2)

conv3 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(pooling2)
pooling3 = keras.layers.MaxPool2D()(conv3)

flatten = keras.layers.Flatten()(pooling3)

dense1 = keras.layers.Dense(256, activation='relu')(flatten)

output = keras.layers.Dense(4, activation='sigmoid')(dense1)

# Creates the model
model = keras.Model(inputs=img_input, outputs=output, name="test_model")
model.summary()

# Compiles the model with an Adam optimizer
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"]
)

# Check that the data is loaded properly (Ensure train[0] and test[0] are valid NumPy arrays)
print("Training data shape:", train[0].shape)
print("Validation data shape:", test[0].shape)

# Fit the model with the training and validation data
history = model.fit(
    train_img,  # Training data (ensure it's a valid NumPy array)
    train_label,  # Training labels (ensure it's a valid NumPy array)
    epochs=10,
    validation_data=(test_img, test_label),  # Tests data (If this doesn't work, try something like test only, without defining the tuples)
    callbacks=[tensorboard_callback, backup],
    verbose=2
)

# Making predictions
# y_pred = model.predict(test_data)  # Use the appropriate test data here
# print("Predictions:", y_pred)


In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
img = files.upload()

In [ ]:
img = cv.imread('/content/drive/MyDrive/Brain_Tumor_Data/Validation/glioma_ver/0001.jpg')
#plt.imshow(img)
#plt.show()
#np.expand_dims(img/255, 0).shape -----> Will give an extra array, or put simply, it will put the entire array into a new array
img = cv.resize(img,(256, 256))
yhat = model.predict(np.expand_dims(img/255, 0)) #We divide img/255 to make sure the values are between 0 and 1, which is easier to work with
yhat = np.squeeze(yhat)


print(yhat)
n=0
mx = yhat[0]

for i in range(len(yhat)):
  if yhat[i] >= mx:
      mx = yhat[i]
      n = i

print(n)

if n == 0:
  print("The given image is of a glioma tumor.")
elif n == 1:
  print("The given image is of a healthy brain.")
elif n == 2:
  print("The given image is of a meningioma tumor.")
elif n == 3:
  print("The given image is of a pituitary tumor.")



In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/Brain_Tumor_Data/logs'

In [ ]:
def check_for_none(data):
  """Checks for None values in the dataset."""
  for i in range(len(data)):
    if data[i] is None:
      print(f"Found None value at index {i}")
      return True  # Return True if None is found
  return False  # Return False if no None is found

if check_for_none(train[0]) or check_for_none(train[1]) or check_for_none(test[0]) or check_for_none(test[1]):
  print("Dataset contains None values. Please check your data loading and preprocessing.")
else:
  print(train[0].shape)
  print(np.any(np.isnan(train[0])))

  # Check a few samples of your data from the generator
for data_batch, labels_batch in data_generator:
    print("Data batch shape:", data_batch.shape)
    print("Labels batch shape:", labels_batch.shape)
    break  # Exit after checking the first batch
